In [10]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_api_client_sdk.exception import AIAPINotFoundException, AIAPIServerException

from sapaicore_utils import (DOCKER_TOKEN,
                             DOCKER_USER,
                             get_conn_details,
                             GITHUB_TOKEN,
                             GITHUB_USER,
                             onboard_repository,
                             onboard_docker,
                             show_docker_registries,
                             show_repositories,)

conn_details = get_conn_details()
ai_core_client = AICoreV2Client(**conn_details)

## Step 0. Onboard a repository and Docker account

### GitHub repository
We'll need a GitHub username and token

In [2]:
# First, let's see which repositories have been onboarded
show_repositories(ai_core_client)

Name: sap-training-ncellini
URL: https://github.com/NCellini/aicore-pipelines
Status: RepositoryStatus.COMPLETED

Name: gsalomone-tf-gpu
URL: https://github.com/gianfranco-s/SAP-aicore-gpu
Status: RepositoryStatus.COMPLETED



In [3]:
PROJECT_NAME = 'gsalomone-tf-gpu'
github_repo_url = 'https://github.com/gianfranco-s/SAP-aicore-gpu'

In [4]:
# If project does not exist, AIAPINotFoundException is raised when `get()` is invoked
try:
    ai_core_client.repositories.get(PROJECT_NAME)
    print('Repository already exists')

except AIAPINotFoundException:
    onboard_repository(github_repo_url,
                    GITHUB_USER,
                    GITHUB_TOKEN,
                    PROJECT_NAME,
                    ai_core_client)
    print('Repository created')

Repository already exists


### Docker account

In [5]:
# First, let's see which registries have been onboarded
show_docker_registries(ai_core_client)

DockerRegistrySecret name: credstutorialrepo-ncellini
DockerRegistrySecret name: gsalomone-docker


In [6]:
docker_registry_name = 'gsalomone-docker'

# If project does not exist, AIAPINotFoundException is raised when `get()` is invoked
try:
    ai_core_client.docker_registry_secrets.get(docker_registry_name)
    print('Docker registry already exists')

except AIAPINotFoundException:
    onboard_docker(docker_registry_name, DOCKER_USER, DOCKER_TOKEN, ai_core_client)
    print('Docker registry created')

Docker registry already exists


## Step 1. Create workspace
Using default resource group because when I tried to create `tf-demo`, I got the message "Resource Group cannot be created for free tier tenant"

In [11]:
try:
    response = ai_core_client.resource_groups.create("tf-demo")

except AIAPIServerException as e:
    print(e)

Failed to post /admin/resourceGroups: Resource Group cannot be created for free tier tenant 
 Status Code: 403, Request ID:None


In [12]:
def show_resource_groups() -> None:
    response = ai_core_client.resource_groups.query()

    for rg in response.resources:
        print(rg.resource_group_id)

show_resource_groups()

default


## Step 2: upload model files to AWS S3
Create bucket
```
aws s3api create-bucket --bucket gsalomone-celestial-bucket --region us-east-1
```

Upload files
```
cd tf_files
aws s3 cp model.h5 s3://gsalomone-celestial-bucket/movie-clf/model/
aws s3 cp max_pad_len.txt s3://gsalomone-celestial-bucket/movie-clf/model/
aws s3 cp label_encoded_classes.npy s3://gsalomone-celestial-bucket/movie-clf/model/
aws s3 cp tokens.json s3://gsalomone-celestial-bucket/movie-clf/model/
```

Check files
```
aws s3 ls s3://gsalomone-celestial-bucket/movie-clf/model/
```

Alternatively, it can be done using the AWS SDK

In [ ]:
import boto3

def upload_file_to_s3(file_dir: str, file_name: str, bucket_name: str, s3_key: str) -> None:
    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_dir + file_name, bucket_name, s3_key + file_name)
        print(f"File uploaded successfully to S3 bucket: {bucket_name} with key: {s3_key}")
    except Exception as e:
        print(f"Error uploading file to S3: {e}")

bucket_name = 'gsalomone-celestial-bucket'
s3_key = 'movie-clf/model/'
file_dir = 'tf_files/'

upload_file_to_s3(file_dir, 'model.h5', bucket_name, s3_key)
upload_file_to_s3(file_dir, 'max_pad_len.txt', bucket_name, s3_key)
upload_file_to_s3(file_dir, 'label_encoded_classes.npy', bucket_name, s3_key)
upload_file_to_s3(file_dir, 'tokens.json', bucket_name, s3_key)


## Step 3: connect AWS S3 to SAP AI Core


In [ ]:
# aws_credentials_file = '/home/gsalomone/Documents/gian-dev_accessKeys.csv'
aws_credentials_file = '/home/gian/gian-dev_accessKeys.csv'

def get_aws_credentials(filepath: str) -> tuple:
    with open(filepath, 'r') as f:
        creds = f.readlines()[1:2][0]
        aws_access_key_id, aws_secret_access_key = creds.split(',')
        return aws_access_key_id, aws_secret_access_key

aws_access_key_id, aws_secret_access_key = get_aws_credentials(aws_credentials_file)
aws_az = 'us-east-1'
object_store_name = 'gs-tf-gpu-tutorial-secret'

In [ ]:
def create_s3_object_store() -> str:
    response = ai_core_client.object_store_secrets.create(
        resource_group = 'default',
        type = "S3",
        name = object_store_name,
        path_prefix = "movie-clf",
        endpoint = f"s3-{aws_az}.amazonaws.com",
        bucket = "gsalomone-celestial-bucket",
        region = aws_az,
        data = {
            "AWS_ACCESS_KEY_ID": aws_access_key_id,
            "AWS_SECRET_ACCESS_KEY": aws_secret_access_key
        }
    )

    return response.message

res = create_s3_object_store()
res

## Step 4: register model as Artifact
Let's follow the tutorial to the letter.

Spoiler: it will not work. **We'll follow step 6 first**

In [ ]:
# Register model as artifact
from ai_core_sdk.models import Artifact

def create_model(model_name: str, object_store_name: str, scenario_id: str, description: str) -> str:
    """Create artifact as model
    
    Keyword arguments:
    model_name -- self explanatory
    object_store_name -- s3 object store
    scenario_id -- taken form the yaml file, found under `scenarios.ai.sap.com/id`
    Return: return_description
    """
    
    response = ai_core_client.artifact.create(
        resource_group = 'default',
        name = model_name,
        kind = Artifact.Kind.MODEL,
        url = f"ai://{object_store_name}/model",
        scenario_id = scenario_id,
        description = description
    )
    return response.message

In [ ]:
try:
    msg = create_model(model_name='gsalomone-model',
                       scenario_id='tf-text-clf',
                       description='Review Classification Model',
                       object_store_name=object_store_name,)

    print(msg)
except Exception as e:
    print(e)

### Step 4 -> Step 6: we need to set up the scenario_id FIRST
It turns out that `scenario_id` is created from the yaml file. So we'll move to step 6 of the tutorial

- Copy [`serving_executable.yaml`](https://raw.githubusercontent.com/sap-tutorials/Tutorials/master/tutorials/ai-core-tensorflow-byod/files/workflow/serving_executable.yaml)
- set `resourcePlan` to `infer.s` which will enable GPU. ATTENTION: free tier only allows for `starter`
- set `imagePullSecrets > name` to the appropriate docker registry secret. In this tutorial you'll find it in variable `docker_registry_name`
- set `containers > image` is set to `"docker.io/DOCKER_USER/movie-review-clf-serve:0.0.1"`

### Step 4 -> Step 7: sync with SAP AI Core
- Push `serving_executable.yaml` to GitHub
- Create the application in SAP AI Core
- Check for typos in yaml file `executables.ai.sap.com/name: "serve-executuable"` -> `executables.ai.sap.com/name: "serve-executable"`


In [ ]:
# Create application
application_name = 'gsalomone-tf-app'
response = ai_core_client.applications.create(
    application_name = application_name,
    revision = "HEAD",
    repository_url = github_repo_url,
    path = "tf-text-classifier"  # path to the yaml file in the repository
)


In [ ]:
response.__dict__

In [ ]:
response = ai_core_client.applications.get_status(application_name=application_name)
print(response.message)

for workflow_sync_status in response.sync_ressources_status:
    print(f'\napplication status: ', workflow_sync_status.status)
    print(f'workflow message: ', workflow_sync_status.message)
    print(f'application name (from yaml file): ', workflow_sync_status.name)

### Step 4 -> Step 4: NOW register model as artifact

In [ ]:
SCENARIO_ID = 'tf-text-clf'  # Remember to take it from the yaml file: `scenarios.ai.sap.com/id`
msg = create_model(model_name='gsalomone-model',
                    scenario_id=SCENARIO_ID,
                    description='Review Classification Model',
                    object_store_name=object_store_name,)

print(msg)


In [ ]:
# Show artifacts

from datetime import datetime
artifacts = ai_core_client.artifact.query(resource_group='default')

artifact_resources = artifacts.resources

for resource in artifact_resources:
    print(f"artifact_id: {resource.id}")
    print(f"created_at: {resource.created_at}")
    print(f"execution_id: {resource.execution_id}")
    print(f"kind: {resource.kind}")
    print(f"name: {resource.name}")
    print(f"scenario_id: {resource.scenario_id}")
    print(f"url: {resource.url}\n")

In [ ]:
# Latest artifact is apparently at the top of the list, so let's define
artifact_id = artifact_resources[0].id
artifact_id

In [ ]:
# Just showing another way to get artifact data
from pprint import pprint

res = ai_core_client.artifact.get(artifact_id=artifact_id, resource_group='default')
pprint(res.__dict__)

## Step 5: set up serving code
```
cd server
sudo docker login docker.io  # Password is the Personal Access Token
sudo docker build -t docker.io/gsalomone/movie-review-clf-serve:0.0.1 .
sudo docker push docker.io/gsalomone/movie-review-clf-serve:0.0.1
```
- Modify requirements to install latest versions
- Use flag `--ignore-installed` in Dockerfile


## Step 8: create configuration

In [ ]:

# Create configuration
from ai_core_sdk.models import InputArtifactBinding

EXECUTABLE_ID = "tf-text-clf-serve"  # It's the (first) `name` key in the yaml file

response = ai_core_client.configuration.create(
    name = "gsalomone-tf-gpu-conf",
    resource_group = "default",
    scenario_id = SCENARIO_ID,
    executable_id = EXECUTABLE_ID,
    input_artifact_bindings = [
        # list
        InputArtifactBinding(key="modelArtifact", artifact_id = artifact_id), # Change artifact id
    ]
)

print(response.__dict__)
config_id = response.__dict__.get('id')

In [ ]:
configurations = ai_core_client.configuration.query(resource_group='default')


for resource in configurations.resources:
    pprint(resource.__dict__)

In [ ]:
# Again, apparently the latest resource appears at the top, so
configuration_id = configurations.resources[0].id
configuration_id

## Step 9: start deployment

In [ ]:
response = ai_core_client.deployment.create(
    resource_group = "default",
    configuration_id = configuration_id
)

In [ ]:
response.__dict__

In [ ]:
deployments = ai_core_client.deployment.query(resource_group="default")

# Following what we've learned about the query method:
deployment_id = deployments.resources[0].id
deployment_id

In [ ]:
# Check deployment status
# This may take 2-3 minutes to change state from UNKNOWN > PENDING > RUNNING.

response = ai_core_client.deployment.get(resource_group="default", deployment_id=deployment_id)

print("Status: ", response.status)
print('*'*80)
pprint(response.__dict__)